# This script takes singerwise masterfiles and finds the stable notes and SDS timestamps, and saves them for downstream tasks

In [2]:
from functions import *
import warnings
warnings.filterwarnings("ignore")

In [3]:
for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:
    
    # Reading masterfile for this singer
    master = pd.read_csv(f'./../../Data/check_output_Jan20/check_output_v2/{singer}_gesture_pitch_pd_with_vel_accln.csv')
    
    # list of audio files we want to process for this singer
    filelist = list(master['filename'].unique())
    
    
    for filename in filelist:
        
        # taking one alap at a time
        aalap = master.loc[master['filename']==filename].reset_index(drop=True)
        
        # ipcdf = Interpolated Pitch Contour DataFrame
        ipcdf = aalap[['time','pitch']].reset_index(drop=True)
        
        # Computing Pitch Class Distribution
        bins = pcd(ipcdf) 
        
        # Splitting contour into Silence Delimited Segments (SDS)
        segments = split_into_segments(ipcdf)
        
        # Finding steady notes
        steady_note_output = find_steady_notes_v2(ipcdf, segments, bins)
        
        # Converting SDS and stable note data to Pandas dataframes
        sds_df = pd.DataFrame(segments,columns=['Start','End'])
        st_df = pd.DataFrame(steady_note_output,columns=['SDS','Start','End','Bin'])
        
        # Interpolating gaps
        new_pitch, merged_st_df = merge_with_interp(ipcdf, st_df)
        ipcdf['new_pitch'] = new_pitch
        
        # Saving dataframes to csv
        sds_df.to_csv(f'./SDS_Dataframes/{filename}',index=False)
        st_df.to_csv(f'./Stable_Notes/{filename}',index=False)
        ipcdf.to_csv(f'./IPCDF/{filename}',index=False)
        